# AE 771 Final
Written by Brendan

## Imported Modules

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ipywidgets as widgets
import time
import math
from IPython.display import display

## Atmo Tables

In [2]:
SIAltTable=[i*1000 for i in [0,1,3,5,10,25,50,75,100,130,160,200,300,400,600,1000]]
SITempTable=[288.15,281.651,268.65,255.65,223.252,221.552,270.65,206.65,195.08,469.27,696.29,845.56,976.01,995.83,999.85,1000]
SIPressTable=[i*101325 for i in [1,0.887,0.66919,0.53313,0.26151,0.025158,0.00078735,0.000020408,0.00000031593,1.2341e-8,2.9997e-9,8.3628e-10,8.6557e-11,1.4328e-11,8.1056e-13,7.4155e-14]]
USAltTable=[i*3.28084 for i in SIAltTable]
USTempTable=[i*1.8 for i in SITempTable]
USPressTable=[i*0.020885434273039 for i in SIPressTable]

AltTable=SIAltTable
TempTable=SITempTable
PressTable=SIPressTable
    
%run RocketEquations.py


## Code

In [3]:
Metric = widgets.Button(description="Run Data")
outputMetric = widgets.Output(layout={'border': '1px solid black'})

def SI(b):
    with outputMetric:
        outputMetric.clear_output()
# Table Selection
        
        AltTable=SIAltTable
        TempTable=SITempTable
        PressTable=SIPressTable
        
# Fuel Selection
        Prop=PropType(Oxidizer.value,Fuel.value)
        r=Prop[0]      # Mass prop ratio
        rV=Prop[1]     # Volume prop ratio
        SG=Prop[2]     # Average specific gravity
        t1=Prop[3]     # Combustion Temperature K
        #cst=Prop[4]    # C Star (speed of sound / thrust coeff)
        MW=Prop[5]     # Molar Mass kg/mol
        Isp=Prop[6]    #Specific Impulse
        k=Prop[7]      # Ratio of Specific Heats
        
# Pre-setting Data Point Arrays
        X=[]
        Y=[]
        X15=[]
        Y15=[]
        
# Constants
        g=9.80665      # m/s^2
        R=8314         # kJ/mol-K
        VCF=0.98       # Velocity Correction
        TCF=0.97       # Thrust Correction
        
# Calculating Required Parameters and Tabulating
        F_IDEAL=Thrust.value  # N
        p1=689475.7*ChamberPressure.value # Pa
        p2=101325*Ambient.value       # Pa
        p3=p2
        ToF=3*60     # seconds
        
                
        dwo=71.1*4.4482216/0.0283         # Weight Density 
        dwf=56.1852*4.4482216/0.0283      # Weight Density 
        denO=dwo/g
        denF=dwf/g
        
        F_TRUE=F_IDEAL*TCF
        p2p1=p2/p1
        
        v2IDEAL=ExhaustVMol(k,R,t1,MW,p2p1)
        v2TRUE=v2IDEAL*VCF
        
        IspIDEAL=SpecificImpulseV(v2IDEAL,g)
        IspTRUE=SpecificImpulseV(v2TRUE,g)
        
        CfIDEAL=CfIdeal(k,p2p1)
        CfTRUE=CfIDEAL*TCF
        
        AtIDEAL=F_IDEAL/(CfIDEAL*p1)
        AtTRUE=F_IDEAL/(CfTRUE*p1)
        AR=AreaRatio(k,p2p1)
        if AR>ARLimit.value:
            AR=ARLimit.value
        A2IDEAL=AtIDEAL*AR
        A2TRUE=AtTRUE*AR
        
        M2=1
        Res=0.0001
        ARint=AR**-1
        
        while ARint<AR:
            M2=M2+Res
            ARint=ExpansionRatio(k,M2)
            
        mdotIDEAL=F_IDEAL/v2IDEAL
        mdotTRUE=F_TRUE/v2TRUE
        
        wdtIDEAL=mdotTRUE*g                  # Total Weight Flow Rate
        wdtTRUE=mdotIDEAL*g  
        wdoIDEAL=OxWdot(wdtIDEAL,r)          # Ox Weight Flow Rate
        wdoTRUE=OxWdot(wdtTRUE,r)
        wdfIDEAL=FuelWdot(wdtIDEAL,r)        # Fuel Weight Flow Rate
        wdfTRUE=FuelWdot(wdtTRUE,r)
        
        wpIDEAL=wdtIDEAL*ToF                 # Prop Weight
        wpTRUE=wdtTRUE*ToF
        woIDEAL=wdoIDEAL*ToF                 # Ox Weight
        woTRUE=wdoTRUE*ToF
        wfIDEAL=wdfIDEAL*ToF                 # Fuel Weight
        wfTRUE=wdfTRUE*ToF
        
        VdoIDEAL=wdoIDEAL/dwo                # Ox Volume Flow Rate
        VdoTRUE=wdoTRUE/dwo
        VdfIDEAL=wdfIDEAL/dwf                # Fuel Volume Flow Rate
        VdfTRUE=wdfTRUE/dwf
        
        VoIDEAL=VdoIDEAL*ToF                 # Ox Volume 
        VoTRUE=VdoTRUE*ToF
        VfIDEAL=VdfIDEAL*ToF                 # Fuel Volume
        VfTRUE=VdfTRUE*ToF
            

        
# Calculating Nozzle Dimensions
        Rt=(AtTRUE/3.141592)**0.5 # m
        Re=Rt*(AR)**0.5           # m
        PL15=L15.value
        L=PL15*((Re-Rt)/math.tan(math.radians(15))) # SCALING FACTOR
        Angle=LARA(PL15,AR)
        AngleN=Angle[0]                 # Degrees
        AngleE=Angle[1]
# Converging Nozzle
        for i in range (-135,-90,1):
            X.append(1.5*Rt*math.cos(math.radians(i)))
            Y.append(1.5*Rt*math.sin(math.radians(i))+1.5*Rt+Rt)
            X15.append(1.5*Rt*math.cos(math.radians(i)))
            Y15.append(1.5*Rt*math.sin(math.radians(i))+1.5*Rt+Rt)

# Throat and Circular Divergence
        for i in range (-90,(AngleN-90),1):
            X.append(0.382*Rt*math.cos(math.radians(i)))
            Y.append(0.382*Rt*math.sin(math.radians(i))+0.382*Rt+Rt)

        for i in range (-90,(15-90),1):    
            X15.append(0.382*Rt*math.cos(math.radians(i)))
            Y15.append(0.382*Rt*math.sin(math.radians(i))+0.382*Rt+Rt)

# Bell Divergence
        Ex=L
        Ey=Re
        Nx=0.382*Rt*math.cos(math.radians(AngleN-90))
        Ny=0.382*Rt*math.sin(math.radians(AngleN-90))+0.382*Rt+Rt
        m1=math.tan(math.radians(AngleN))
        m2=math.tan(math.radians(AngleE))
        C1=Ny-m1*Nx
        C2=Ey-m2*Ex
        Qx=(C2-C1)/(m1-m2)
        Qy=(m1*C2-m2*C1)/(m1-m2)

        for t in range (0,1000,1):
            t=t/1000
            X.append(Qx+(1-t)**2*(Nx-Qx)+t**2*(Ex-Qx))
            Y.append(Qy+(1-t)**2*(Ny-Qy)+t**2*(Ey-Qy))
            
# Combustion Chamber Work
        Dt=Rt*2
        Dc=Dt*2
        Rc=Dt
        Ac=0.5*math.pi*Rc**2
        Beta=45
        
        Lst=Lstar.value #m
        Vchamber=Lst*AtTRUE
        Lconv=-Rc/math.tan(math.radians(Beta))
        Vconv=(1/3)*math.pi*(Rc**2+Rc*Rt+Rt**2)*Lconv
        
        Vcyl=Vchamber-Vconv
        Lcyl=Vcyl/Ac
        
        Lchamber=Lcyl+Lconv
        
        CDXinit=-Lchamber
        CDX=[CDXinit]
        CDY=[Rc]
        CDXcrv=X[0]-(Rc-Y[0])
        
        while CDXinit<CDXcrv:
            CDXinit=CDXinit+0.00001
            CDY.append(Dt)
            CDX.append(CDXinit)
            
        CDYinit=Rc
        
        while CDXinit<X[0]:
            CDXinit=CDXinit+0.00001
            CDYinit=CDYinit-0.00001
            CDY.append(CDYinit)
            CDX.append(CDXinit)
        
# Plotting the Bell Nozzle 
        NegY=[i*-1 for i in Y]
        NegCDY=[i*-1 for i in CDY]
        plt.xlabel('Length in Meters')
        plt.ylabel('Width in Meters')
        plt.plot(X,Y,color='tab:gray')
        plt.plot(CDX,CDY,color='tab:gray')
        plt.plot(X,NegY,color='tab:gray')
        plt.plot(CDX,NegCDY,color='tab:gray')
        plt.xlim([-3.76631*Re*.75,7.0433*Re*.75])
        plt.ylim([-3.48458*Re*.75,3.48458*Re*.75])
        plt.title('Rao Nozzle Plot')
        plt.show()

# 15 Degree Half Angle Divergence
        Nx15=0.382*Rt*math.cos(math.radians(15-90))
        Ny15=0.382*Rt*math.sin(math.radians(15-90))+0.382*Rt+Rt

        xtemp=Nx15
        while math.tan(math.radians(15))*xtemp+Ny15 < Re:
            xtemp=xtemp+0.00001
            Y15.append(math.tan(math.radians(15))*xtemp+Ny15-math.tan(math.radians(15))*Nx15)
            X15.append(xtemp)
            

# Plotting 15 degree Half Angle Nozzle
        NegY15=[i*-1 for i in Y15]
        plt.xlabel('Length in Meters')
        plt.ylabel('Width in Meters')
        plt.plot(X15,Y15,color='tab:gray')
        plt.plot(CDX,CDY,color='tab:gray')
        plt.plot(X15,NegY15,color='tab:gray')
        plt.plot(CDX,NegCDY,color='tab:gray')
        plt.xlim([-3.76631*Re*.75,7.0433*Re*.75])
        plt.ylim([-3.48458*Re*.75,3.48458*Re*.75])
        plt.title('15 Degree Angle Nozzle Plot')
        plt.show()
        
# Combustion Instability
        RadialL=2*(Dc)
        TanL=2*math.pi*(Dc)
        AxialL=2*(Lchamber)
        
        a=(k*t1*R/MW)**0.5
        
        RadialF=a/RadialL
        TanF=a/TanL
        AxialF=a/AxialL
        
# Injector Design
        #Injector
#Metric Conversions
        Mdoto_A = Wdoto_A*4.44822/9.81 #Mass Flowrate of Oxidizer | kg/s
        Mdotf_A = Wdotf_A*4.44822/9.81 #Mass Flowrate of Fuel | kg/s
        ρom=ρo*157.087                  #Liquid Weight Density of Oxygen (Oxidizer) | N/m^3
        ρfm=ρf*157.087                  #Liquid Weight Density of Hydrogen (Fuel) | N/m^3
        P1m=P1                         #Chamber Pressure |Pa
        dP = 0.2*P1m                    #Change in pressure | Pa
        Cd = 0.9                       #Discharge Coefficient of Injector | Table 8-2 | Unitless
        Dcm = Dc*25.4                   #Diameter of the Chamber | mm
        AC=(np.pi*Dcm**2)/4             #Calculating the Area of the Chamber | mm**2
        θI = 25                        #Injector Angle | deg | Pg 280

        A_IOx = (Mdoto_A)/(Cd*((2*ρo*dP)**(1/2))) #Area of Oxidizer Injector
        A_IF = (MdotF_A)/(Cd*((2*ρf*dP)**(1/2)))   #Area of Fuel Injector
        Ao=A_IOx*1000*1000                         #Converting m**2 to mm**2
        AF=A_IF*1000*1000                          #Converting m**2 to mm**2
        HoleDiam=0.2                             #Diameter of the holes | mm
        HoleSize=(np.pi*HoleDiam**2)/4             #Calculating the Area of the Holes mm**2
        Oxnum=Ao/HoleSize                           #Number of Oxidizer holes
        Fnum=AF/HoleSize                           #Number of Fuel holes
        Treelines = 40                             #Number of Lines of Fuel & Oxy
        TreeSpace = (Dc/2)/Treelines               #Spacing between lines | mm
        TreeComp = 5                               #How much space a doublet takes up from outer radii | mm

#Loop to Calculate the Layout of the Injector
        Ring = 1
        OxArea = 0    #Oxidizer
        FuelArea = 0     #Fuel
        while Ring <= Treelines:
            TreeC = 2*np.pi*TreeSpace*Ring #Circumference of the ring
            RingDoubles = TreeC//TreeComp
            AngleSpace = 360//RingDoubles
            if (Ring%2 == 0):
                AreaAdd = RingDoubles*HoleSize
                OxArea = OxArea+(AreaAdd*2)
            else:
                AreaAdd = RingDoubles*HoleSize
                FuelArea = FuelArea+(AreaAdd*2)
        
    #print("For Ring", Ring,"there are",RingDoubles,"doublets with an angle of:",AngleSpace)
    Ring=Ring+1
    
#Estiamtion Calculation for Ozidizer Hole Variance
ARatio=Ao/FuelArea
D_OxyNew=((HoleSize*ARatio*4)/np.pi)**(1/2) #New Recommended Diameter of the Oxidizer holes
A_OxyNew=FuelArea/HoleSize*(D_OxyNew**2*np.pi/4)
        
# Launch Altitudes and Fuel Parameters
        MassI=1500000*0.4535924 # kg
        MassS=160000*0.4535924  
        MassP=16500*0.4535924   
        
        SMR=MassS/MassI
        PMR=MassP/MassI
        
        #SSTO
        dV=IspTRUE*g*math.log(1/(SMR+PMR))
        
        R=6378
        Ro=R+100*1.609344
        
        mu=3.986e5
        SMEo=-mu/(2*Ro)
        
        
        Vo=math.sqrt(2*(SMEo+mu/Ro))
        Ve=math.sqrt(2*(mu/Ro))
        
        #multi-stages
        dV2=2*IspTRUE*g*math.log(1/(SMR+PMR))
        dV3=3*IspTRUE*g*math.log(1/(SMR+PMR))
        
# Part C calculations
        Tlox=110       # Lox temp 
        Klox=152e-3  # Lox conductivity
        Tg=3033        # Temp of gas
        Twg=900        # Temp of gas wall
        rho=1141         # Specific gravity water
        Visc=195e-6    # Viscosity
        SHlox=5.01        # Specific lox
        PW=6.35/1000     # Passage width
        PH=2.54/1000      # Passage Height
        mdot=0.5     # water mas flow
        Km=12          # Material conductivity
        hg=1500
        
        PA=PW*PH       # Passage Area
        PP=(PW+PH)*2   # Passage Perimeter
        PD=4*(PA/PP)   # Passage Diameter
        
        Vlox=mdot/(rho*PA)
        hl=(0.023*SHlox*(mdot/PA)*(PD*Vlox*rho/Visc)**(-0.2)*
            (Visc*SHlox/Klox)**(-2/3))
        q=hg*(Tg-Twg)
        Twl=q/hl+Tlox
        twall=-Km*(Twl-Twg)/q
        
        
        print('---------------------------------------------------------')
        print('------------------Question A Calculations----------------')
        print('---------------------------------------------------------\n')
        
        print('Value                      Ideal         Actual')
        print('Exit Velocity:             ','{0:.0f}'.format(v2IDEAL),'(m/s)    '
              '{0:.0f}'.format(v2TRUE),'(m/s)\n')
        print('Specific Impulse:        ','{0:.1f}'.format(IspIDEAL),'(s)    '
              '{0:.1f}'.format(IspTRUE),'(s)\n')
        print('Thrust Coeff:             ','{0:.2f}'.format(CfIDEAL),'(~)    '
              '{0:.2f}'.format(CfTRUE),'(~)\n')
        print('Throat Radius:             ','{0:.3f}'.format(Rt),'(m)      '
              '{0:.3f}'.format(Rt),'(m)\n')
        print('Area Ratio:             ','{0:.2f}'.format(AR),'(~)      '
              '{0:.2f}'.format(AR),'(~)\n')
        print('Exit Area:                 ','{0:.2f}'.format(A2IDEAL),'(m^2)    '
              '{0:.2f}'.format(A2TRUE),'(m^2)\n')
        print('Exit Mach:                 ','{0:.2f}'.format(M2),'(~)        '
              '{0:.2f}'.format(M2),'(~)\n')
        print('Propellant Weight Flow Rate:','{0:.2f}'.format(wdtIDEAL),'(N/s)    '
              '{0:.2f}'.format(wdtTRUE),'(N/s)\n')
        print('Propellant Volume Flow Rate:','{0:.3f}'.format(VdoIDEAL+VdfIDEAL),'(m^3/s)    '
              '{0:.3f}'.format(VdoTRUE+VdfTRUE),'(m^3/s)\n')
        print('Oxidizer Weight Flow Rate: ','{0:.2f}'.format(wdoIDEAL*0.22480894387096),'(lbf/s)    '
              '{0:.2f}'.format(wdoTRUE*0.22480894387096),'(lbf/s)\n')
        print('Fuel Weight Flow Rate:     ','{0:.2f}'.format(wdfIDEAL*0.22480894387096),'(lbf/s)    '
              '{0:.2f}'.format(wdfTRUE*0.22480894387096),'(lbf/s)\n')
        print('Oxidizer Volume Flow Rate: ','{0:.6f}'.format(VdoIDEAL*35.314666572222),'(ft^3/s)    '
              '{0:.4f}'.format(VdoTRUE*35.314666572222),'(ft^3/s)\n')
        print('Fuel Volume Flow Rate:     ','{0:.6f}'.format(VdfIDEAL*35.314666572222),'(ft^3/s)    '
              '{0:.4f}'.format(VdfTRUE*35.314666572222),'(ft^3/s)\n')
        print('Oxidizer Weight:           ','{0:.0f}'.format(woIDEAL),'(N)     '
              '{0:.0f}'.format(woTRUE),'(N)\n')
        print('Fuel Weight:               ','{0:.0f}'.format(wfIDEAL),'(N)     '
              '{0:.0f}'.format(wfTRUE),'(N)\n')
        print('Oxidizer Volume:           ','{0:.4f}'.format(VoIDEAL),'(m^3)    '
              '{0:.4f}'.format(VoTRUE),'(m^3)\n')
        print('Fuel Volume:               ','{0:.4f}'.format(VfIDEAL),'(m^3)    '
              '{0:.4f}'.format(VfTRUE),'(m^3)\n')
        print('---------------------------------------------------------')
        print('---------------------Question A Designs------------------')
        print('---------------------------------------------------------\n')
        print('I have recorded Figure 3-13 into a matrix in the support code') 
        print('by drawing reference to the individual curves based on Area Ratio') 
        print('and the %15 degree value. At this point, no calculation is made') 
        print('and the results are output automatically.\n')
        print('Theta n:',AngleN,'(deg)\n')
        print('Theta e:',AngleE,'(deg)\n')
        print('\n-----------------Combustion Chamber---------------------\n')
        print('L*:',Lst,'(m)')
        print('Converging Duct Length:','{0:.5f}'.format(-CDXcrv),'(m)')
        print('Cylindrical Duct Length:',Lcyl,'(m)')
        print('Chamber Diameter:','{0:.7f}'.format(Dc),'(m)\n')
        print('\n------------------------Frequencies---------------------\n')        
        print('Radial Resonance Frequency:','{0:.2f}'.format(RadialF),'(Hz)\n')
        print('Tangential Resonance Frequency:','{0:.2f}'.format(TanF),'(Hz)\n')
        print('Axial Resonance Frequency:','{0:.2f}'.format(AxialF),'(Hz)\n')
        
        print('\n---------------------------------------------------------')
        print('---------------------Question B---------------------------')
        print('---------------------------------------------------------\n')
        print('\n----------------------Mass Fraction----------------------\n')
        print('Structural Mass Fraction:',round(SMR,3),'(~)\n')
        print('Payload Mass Fraction:',round(PMR,3),'(~)\n')
        print('\n-------------------Velocity Increment--------------------\n')
        print('Velocity Increment 1 Stage:',round(dV,1),'(m/s)\n')
        print('\n-----------------Orbital Velocity Req.-------------------\n')
        print('Circular Orbit:',round(Vo*1000,2),'(m/s)\n')
        print('Escape Trajectory:',round(Ve*1000,2),'(m/s)\n')
        print('The vehicle cannot attain either velocity.\n')
        print('\n------------------Multi Stage Analysis-------------------\n')
        print('Velocity Increment 2 Stage:',round(dV2,1),'(m/s)\n')
        print('Velocity Increment 3 Stage:',round(dV3,1),'(m/s)\n')
        
        print('\n---------------------------------------------------------')
        print('---------------------Question C---------------------------')
        print('---------------------------------------------------------\n')
        print('Liquid Film Coefficient:',round(hl,1),'(W/m^2-K)')
        print('Steady State Heat Flux:',q,'(W/m^2)')
        print('Coolant Side Wall Temp.:',round(Twl,1),'(K)')
        print('Wall Thickness:',round(twall*1000,2),'(mm)\n')
        print('\n---------------------------------------------------------')
        print('---------------------Question D---------------------------')
        print('---------------------------------------------------------\n')
        print('Q.1')
        print('a)')
        print('   1) SSME Thrust: 1.86  MN')
        print('   2) OMS Thrust:  26.7  kN')
        print('   3) Primary RCS: 3.87  kN')
        print('      Vernier RCS: 106    N')
        print('   4) SRB Thrust:  12.5  MN')
        print('b) The OMS and the RCS systems used the same MMH/N2O4 fuel/ox.')
        print('c) The SRBs burned for 124 sec. while the SSMEs burned for 480 sec.')
        print('\n---------------------------------------------------------\n')
        print('Q.2')
        print('A hybrid rocket burns a liquid oxidizer with a solid fuel while an')
        print('inverse hybrid rocket buns a solid oxidizer with liquid fuel.')
        print('\n---------------------------------------------------------\n')
        print('Q.3')
        print('a)Increases')
        print('b)')
        print('c)Decreases')
        print('d)')
        print('e)')
        print('f)')
        print('\n---------------------------------------------------------\n')
        print('Q.4')
        print('\n---------------------------------------------------------\n')
        print('Q.5')
        print('a)')
        print('b) Pyrotechnic and pyrogen.')
        
Metric.on_click(SI)

## GUI

In [4]:
Oxidizer=widgets.Dropdown(
    options=['N/A','Oxygen','Flourine','Nitrogen Tetroxide','Nitric Acid','Hydrogen Peroxide'],
    value='Oxygen',
    description='Oxidizer:',
    disabled=False,
)

Fuel=widgets.Dropdown(
    options=['Air','Methane','Hydrazine','Hydrogen','RP-1','UDMH','UDMH/Hydrazine','MMH'],
    value='RP-1',
    description='Fuel:',
    disabled=False,
)

Thrust=widgets.FloatText(
    value=6672332.4,
    description='Thrust (N): ',
    disabled=False,
)

ChamberPressure=widgets.FloatText(
    value=10,
    description='Chamber 100 psi: ',
    disabled=False,
)

L15=widgets.FloatText(
    value=0.8,
    description='% of 15 deg: ',
    disabled=False,
)

Ambient=widgets.FloatText(
    value=1,
    description='Ambient: ',
    disabled=False,
)

Lstar=widgets.FloatText(
    value=5,
    description='L*: ',
    disabled=False,
)

ARLimit=widgets.FloatText(
    value=100,
    description='Max AR: ',
    disabled=False,
)

DispWindows1=widgets.HBox([Oxidizer,Fuel],
                        layout=widgets.Layout(overflow='visible',
                        align_items='center',
                        align_contents='center',)
)

DispWindows2=widgets.HBox([Thrust,ChamberPressure],
                        layout=widgets.Layout(overflow='visible',
                        align_items='center',
                        align_contents='center',)
)

DispWindows3=widgets.HBox([L15,Ambient],
                        layout=widgets.Layout(overflow='visible',
                        align_items='center',
                        align_contents='center',)
)

DispWindows4=widgets.HBox([ARLimit,Lstar],
                        layout=widgets.Layout(overflow='visible',
                        align_items='center',
                        align_contents='center',)
)
display(DispWindows1)
display(DispWindows2)
display(DispWindows3)
display(DispWindows4)
display(Metric, outputMetric)




Button(description='Run Data', style=ButtonStyle())

Output(layout=Layout(border='1px solid black'))